<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego 
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej" 
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

In this lab, you will implement some of the techniques discussed in the lecture.

Below you are given a solution to the previous scenario. Note that it has two serious drawbacks:
 * The output predictions do not sum up to one (i.e. it does not return a distribution) even though the images always contain exactly one digit.
 * It uses MSE coupled with output sigmoid which can lead to saturation and slow convergence 

**Task 1.** Use softmax instead of coordinate-wise sigmoid and use log-loss instead of MSE. Test to see if this improves convergence. Hint: When implementing backprop it might be easier to consider these two function as a single block and not even compute the gradient over the softmax values. 

**Task 2.** Implement L2 regularization and add momentum to the SGD algorithm. Play with different amounts of regularization and momentum. See if this improves accuracy/convergence.

**Task 3 (optional).** Implement Adagrad, dropout and some simple data augmentations (e.g. tiny rotations/shifts etc.). Again, test to see how these changes improve accuracy/convergence.

**Task 4.** Try adding extra layers to the network. Again, test how the changes you introduced affect accuracy/convergence. As a start, you can try this architecture: [784,100,30,10]


In [25]:
import random
import numpy as np
from torchvision import datasets, transforms

In [26]:
# Let's read the mnist dataset

def load_mnist(path='.'):
    train_set = datasets.MNIST(path, train=True, download=True)
    x_train = train_set.data.numpy()
    _y_train = train_set.targets.numpy()
    
    test_set = datasets.MNIST(path, train=False, download=True)
    x_test = test_set.data.numpy()
    _y_test = test_set.targets.numpy()
    
    x_train = x_train.reshape((x_train.shape[0],28*28)) / 255.
    x_test = x_test.reshape((x_test.shape[0],28*28)) / 255.

    y_train = np.zeros((_y_train.shape[0], 10))
    y_train[np.arange(_y_train.shape[0]), _y_train] = 1
    
    y_test = np.zeros((_y_test.shape[0], 10))
    y_test[np.arange(_y_test.shape[0]), _y_test] = 1

    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_mnist()

In [27]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    # Derivative of the sigmoid
    return sigmoid(z)*(1-sigmoid(z))

def softmax(z):
    # Numericaly stable
    exps = np.exp(z - np.max(z))
    sum = np.sum(exps, axis=0)
    res = exps / sum
    return res

In [28]:
class Network(object):
    def __init__(self, sizes):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
    def feedforward(self, a):
        # Run the network on a batch
        a = a.T
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.matmul(w, a)+b)
        return a
    
    def update_mini_batch(self, mini_batch, eta):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch which is as in tensorflow API.
        # eta is the learning rate      
        nabla_b, nabla_w = self.backprop(mini_batch[0].T,mini_batch[1].T)
            
        self.weights = [w-(eta/len(mini_batch[0]))*nw 
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch[0]))*nb 
                       for b, nb in zip(self.biases, nabla_b)]
        
    def backprop(self, x, y):
        # For a single input (x,y) return a pair of lists.
        # First contains gradients over biases, second over weights.
        g = x
        gs = [g] # list to store all the gs, layer by layer
        fs = [] # list to store all the fs, layer by layer
        for b, w in zip(self.biases, self.weights):
            f = np.dot(w, g)+b
            fs.append(f)
            g = sigmoid(f)
            gs.append(g)
        # backward pass <- both steps at once
        dLdg = self.cost_derivative(gs[-1], y)
        dLdfs = []
        for w,g in reversed(list(zip(self.weights,gs[1:]))):
            dLdf = np.multiply(dLdg,np.multiply(g,1-g))
            dLdfs.append(dLdf)
            dLdg = np.matmul(w.T, dLdf)
        
        dLdWs = [np.matmul(dLdf,g.T) for dLdf,g in zip(reversed(dLdfs),gs[:-1])] 
        dLdBs = [np.sum(dLdf,axis=1).reshape(dLdf.shape[0],1) for dLdf in reversed(dLdfs)] 
        return (dLdBs,dLdWs)

    def evaluate(self, test_data):
        # Count the number of correct answers for test_data
        pred = np.argmax(self.feedforward(test_data[0]),axis=0)
        corr = np.argmax(test_data[1],axis=1).T
        return np.mean(pred==corr)
    
    def cost_derivative(self, output_activations, y):
        return (output_activations-y) 
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        for j in range(epochs):
            for i in range(x_train.shape[0] // mini_batch_size):
                x_mini_batch = x_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                y_mini_batch = y_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                self.update_mini_batch((x_mini_batch, y_mini_batch), eta)
            if test_data:
                print("Epoch: {0}, Accuracy: {1}".format(j, self.evaluate((x_test, y_test))))
            else:
                print("Epoch: {0}".format(j))

In [29]:
class MyNetwork(object):
    def __init__(self, sizes, lambda_l2=0.0, momentum=0.0, dropout_p=0.0):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.lambda_l2 = lambda_l2
        self.momentum = momentum
        self.dropout_p = dropout_p
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
        self.weights_momentum = [np.zeros_like(x) for x in self.weights]
        self.biases_momentum = [np.zeros_like(x) for x in self.biases]
    
    def activation(self, z, l):
        if l == (self.num_layers - 1):
            return softmax(z)
        else:
            return sigmoid(z)
    
    def feedforward(self, a):
        # Run the network on a batch
        a = a.T
        for l, b, w in zip(range(1, self.num_layers), self.biases, self.weights):
            if l < self.num_layers - 1:
                a = self.activation((1.0-self.dropout_p)*(np.matmul(w, a)+b), l)
            else:
                a = self.activation(np.matmul(w, a)+b, l)
        return a
    
    def update_mini_batch(self, mini_batch, eta):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch which is as in tensorflow API.
        # eta is the learning rate      
        nabla_b, nabla_w = self.backprop(mini_batch[0].T,mini_batch[1].T)
        
        ### Momentum equation for parameter p
        ### p_(t+1) = p_t + m_(t+1)
        ### m_(t+1) = lambda_momentum * m_t - eta * gradient(p_t)
        self.weights_momentum = [(self.momentum*wm)-(eta/len(mini_batch[0]))*nw 
                                 for wm, w, nw in zip(self.weights_momentum, self.weights, nabla_w)]
        self.biases_momentum = [(self.momentum*bm)-(eta/len(mini_batch[0]))*nb 
                                for bm, b, nb in zip(self.biases_momentum, self.biases, nabla_b)]
                            
        self.weights = [w+wm for w, wm in zip(self.weights, self.weights_momentum)]
        self.biases = [b+bm for b, bm in zip(self.biases, self.biases_momentum)]
        
    def backprop(self, x, y):
        # For a single input (x,y) return a pair of lists.
        # First contains gradients over biases, second over weights.
        ### Dropout
        dropout = [np.random.choice([0,1], size=(k,x.shape[1]), p=[self.dropout_p, 1-self.dropout_p]) for k in self.sizes[:-1]]
        dropout.append(np.full((self.sizes[-1], x.shape[1]), 1))

        g = np.multiply(dropout[0], x)
        gs = [g] # list to store all the gs, layer by layer
        fs = [] # list to store all the fs, layer by layer
        for l, b, w in zip(range(1, self.num_layers), self.biases, self.weights):
            f = np.multiply(dropout[l], (np.dot(w, g)+b))
            fs.append(f)
            g = self.activation(f, l)
            gs.append(g)
        # backward pass <- both steps at once
        ### we know that [logloss(softmax)]' ~= [mse]'
        ### change dLdg to dLdf -> considering logloss and softmax as a single block
        ### the rest of dLdf calculations can be left as they were
        dLdf = self.cost_derivative(gs[-1], y)
        dLdfs = []
        for l,w,g in reversed(list(zip(range(1, self.num_layers), self.weights, gs[1:]))):
            if l < (self.num_layers - 1):
                dLdf = np.multiply(np.multiply(dLdg, np.multiply(g,1-g)), dropout[l])
            dLdfs.append(dLdf)
            dLdg = np.matmul(w.T, dLdf)
        
        dLdWs = [np.matmul(dLdf,g.T) for dLdf,g in zip(reversed(dLdfs),gs[:-1])] 
        dLdBs = [np.sum(dLdf,axis=1).reshape(dLdf.shape[0],1) for dLdf in reversed(dLdfs)] 

        ### L2 regularization
        ### loss = old_loss + lambda_l2 * sum(weights^2)
        dLdWs = [dLdW + (self.lambda_l2 * w) for dLdW, w in zip(dLdWs, self.weights)]

        return (dLdBs,dLdWs)

    def evaluate(self, test_data):
        # Count the number of correct answers for test_data
        pred = np.argmax(self.feedforward(test_data[0]),axis=0)
        corr = np.argmax(test_data[1],axis=1).T
        return np.mean(pred==corr)
    
    def cost_derivative(self, output_activations, y):
        return (output_activations-y) 
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        for j in range(epochs):
            for i in range(x_train.shape[0] // mini_batch_size):
                x_mini_batch = x_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                y_mini_batch = y_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                self.update_mini_batch((x_mini_batch, y_mini_batch), eta)
            if test_data:
                print("Epoch: {0}, Accuracy: {1}".format(j, self.evaluate((x_test, y_test))))
            else:
                print("Epoch: {0}".format(j))

In [30]:
%%time

network = Network([784,30,10])
network.SGD((x_train, y_train), epochs=10, mini_batch_size=100, eta=3.0, test_data=(x_test, y_test))

Epoch: 0, Accuracy: 0.6442
Epoch: 1, Accuracy: 0.7708
Epoch: 2, Accuracy: 0.7934
Epoch: 3, Accuracy: 0.8056
Epoch: 4, Accuracy: 0.814
Epoch: 5, Accuracy: 0.8202
Epoch: 6, Accuracy: 0.8252
Epoch: 7, Accuracy: 0.8278
Epoch: 8, Accuracy: 0.8309
Epoch: 9, Accuracy: 0.8337
CPU times: user 2min 58s, sys: 2min 26s, total: 5min 25s
Wall time: 31.5 s


In [33]:
%%time

network = MyNetwork([784,30,10], lambda_l2=0.01, momentum=0.5)
network.SGD((x_train, y_train), epochs=10, mini_batch_size=100, eta=1.0, test_data=(x_test, y_test))

Epoch: 0, Accuracy: 0.8877
Epoch: 1, Accuracy: 0.9119
Epoch: 2, Accuracy: 0.9248
Epoch: 3, Accuracy: 0.9336
Epoch: 4, Accuracy: 0.9391
Epoch: 5, Accuracy: 0.9424
Epoch: 6, Accuracy: 0.9445
Epoch: 7, Accuracy: 0.9471
Epoch: 8, Accuracy: 0.9495
Epoch: 9, Accuracy: 0.9504
CPU times: user 9min 47s, sys: 9min 34s, total: 19min 22s
Wall time: 2min


In [38]:
%%time

network = MyNetwork([784,100,30,10], lambda_l2=0.01, momentum=0.1, dropout_p=0.1)
network.SGD((x_train, y_train), epochs=10, mini_batch_size=100, eta=3.0, test_data=(x_test, y_test))

Epoch: 0, Accuracy: 0.8962
Epoch: 1, Accuracy: 0.9198
Epoch: 2, Accuracy: 0.9349
Epoch: 3, Accuracy: 0.9403
Epoch: 4, Accuracy: 0.9465
Epoch: 5, Accuracy: 0.9097
Epoch: 6, Accuracy: 0.9221
Epoch: 7, Accuracy: 0.9616
Epoch: 8, Accuracy: 0.9588
Epoch: 9, Accuracy: 0.9591
CPU times: user 10min 46s, sys: 10min 12s, total: 20min 59s
Wall time: 1min 57s
